In [1]:
import cv2
import torch
from ultralytics import YOLO

C:\Users\Priyansh Sethi\anaconda3\anaconda_3\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
def natural_sort_key(filename):
    """Extract numerical values from filenames for correct sorting."""
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', filename)]

In [3]:
def detect_objects(model, image_path):
    image = cv2.imread(image_path)
    results = model(image)  # Perform object detection
    return results, image


In [4]:
def get_ceiling_floor_coordinates(results, floor_model):
    ceiling_y = None  # Initialize
    floor_y = None  # Initialize
    best_ceiling = None
    best_floor = None

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())  # Bounding box coords
            confidence = float(box.conf[0].cpu().numpy())  # Confidence score
            class_id = int(box.cls[0].cpu().numpy())  # Class ID
            
            class_name = floor_model.names.get(class_id, "Unknown")

            if class_name == "ceiling":
                if best_ceiling is None or confidence > best_ceiling[1]:
                    best_ceiling = ((x1, y1, x2, y2), confidence)

            if class_name == "floor":
                if best_floor is None or confidence > best_floor[1]:
                    best_floor = ((x1, y1, x2, y2), confidence)

    ceiling_mid, floor_mid = None, None  

    if best_ceiling:
        (_, ceiling_y1, _, ceiling_y2), _ = best_ceiling
        ceiling_y = (ceiling_y1 + ceiling_y2)// 2  # Midpoint

    if best_floor:
        (_, floor_y1, _, floor_y2), _ = best_floor
        floor_y = (floor_y1 + floor_y2)// 2  # Midpoint

    return ceiling_y, floor_y


In [5]:
def get_floor_coordinates(results):
    """Extract the top Y-coordinate and bounding box of the floor"""
    floor_y_top = None
    floor_box = None
    
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls.cpu().numpy().item())  # Get class ID
            x_min, y_min, x_max, y_max = map(int, box.xyxy[0])  # Bounding box
            
            if class_id == 1:  # Assuming class 1 is "floor"
                floor_y_top = y_min  # Top Y-coordinate of floor
                floor_box = (x_min, y_min, x_max, y_max)
    
    return floor_y_top, floor_box

In [6]:
def get_propeller_coordinates(results):
    """Extract the bottom Y-coordinates and bounding boxes of propellers"""
    propeller_boxes = []  # Store bounding boxes of detected propellers
    propeller_y_bottoms = []  # Store Y-bottom of propellers
    
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls.cpu().numpy().item())  # Get class ID
            x_min, y_min, x_max, y_max = map(int, box.xyxy[0])  # Bounding box
            
            if class_id == 0:  # Assuming class 0 is "propeller"
                propeller_y_bottoms.append(y_max)  # Store bottom Y-coordinate
                propeller_boxes.append((x_min, y_min, x_max, y_max))
    
    return propeller_y_bottoms, propeller_boxes

In [7]:
def detect_height_bottle(image_path, model):
    """Detect Coca-Cola bottles and return their bounding box height in pixels."""
    image = cv2.imread(image_path)
    results = model(image)
    coke_boxes = []
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0]  # Bounding box coordinates
            height_pixels = y2 - y1  # Bottle height in pixels
            real_height_cm = 26.3  # Known height of Coca-Cola bottle
            coke_boxes.append((x1, y1, x2, y2))
            pixel_to_cm_ratio = real_height_cm / height_pixels
            estimated_real_height = height_pixels * pixel_to_cm_ratio
            
            return height_pixels, estimated_real_height
    

    return None, None  # If no bottle is detected

In [8]:
def calculate_height(ceiling_y, floor_y, calibration_obj):
    """Calculate ceiling height using a calibration object (Coca-Cola bottle)."""
    if ceiling_y is None or floor_y is None:
        print("Error: Ceiling or Floor not detected!")
        return None  # Avoid TypeError

    pixel_height = floor_y - ceiling_y

    if calibration_obj:
        real_height, obj_pixel_height = calibration_obj
        pixel_to_cm_ratio = real_height / obj_pixel_height
        return pixel_height * pixel_to_cm_ratio
    
    return pixel_height  # Return pixel height if no calibration object


In [9]:
def draw_detections(image, propeller_boxes, floor_box, calibration_obj, floor_results):
    best_ceiling = None
    best_floor = None  # Ensure variables are initialized
    for i, (x1, y1, x2, y2) in enumerate(propeller_boxes):  # Unpacking tuple directly
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])  # Ensure integer values
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)     
        # Calculate height from floor to propeller
        pixel_height = y1 - floor_box[1]
        real_height = calculate_height(y1, floor_box[1], calibration_obj)
        height_text = f"Height: {real_height:.2f} cm"
        # Draw text label
        cv2.putText(image, height_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    # Label the calibration object
    if calibration_obj:
        real_height, obj_pixel_height = calibration_obj
        cal_text = f"Calibration: {real_height:.2f} cm ({obj_pixel_height} px)"
        cv2.putText(image, cal_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
    # Process results
    for result in floor_results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())  # Bounding box coordinates
            confidence = float(box.conf[0].cpu().numpy())  # Confidence score
            class_id = int(box.cls[0].cpu().numpy())  # Class ID

            # Get class name correctly
            class_name = floor_model.names.get(class_id, "Unknown")  # Ensure we get the correct class label

            # Store the most confident "ceiling" and "floor" detection
            if class_name == "ceiling":
                if best_ceiling is None or confidence > best_ceiling[1]:
                    best_ceiling = ((x1, y1, x2, y2), confidence, class_name)

            if class_name == "floor":
                if best_floor is None or confidence > best_floor[1]:
                    best_floor = ((x1, y1, x2, y2), confidence, class_name)
    # Draw only the best detections on the image
    for best in [best_ceiling, best_floor]:
        if best is not None:  # Ensure best_ceiling and best_floor are not None
            (x1, y1, x2, y2), confidence, class_name = best
            label = f'{class_name}: {confidence:.2f}'
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    return image


In [10]:
def draw_annotations(image, coke_boxes, ceiling_y, floor_y, pixel_to_cm_ratio=None):
    """Draws bounding boxes, vertical line, and labels on the image."""
    annotated_image = image.copy()
    
    # Draw Coca-Cola bounding boxes
    for (x1, y1, x2, y2, height_pixels) in coke_boxes:
        cv2.rectangle(annotated_image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box
        cv2.putText(annotated_image, f"{height_pixels}px", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Draw vertical line from floor to ceiling
    if ceiling_y is not None and floor_y is not None:
        mid_x = image.shape[1] // 2  # Center x-coordinate
        cv2.line(annotated_image, (mid_x, ceiling_y), (mid_x, floor_y), (0, 0, 255), 2)  # Red line
        
        # Calculate real-world ceiling height if calibration object exists
        pixel_height = floor_y - ceiling_y
        if pixel_to_cm_ratio:
            real_ceiling_height = pixel_height * pixel_to_cm_ratio
            height_text = f"Height: {real_ceiling_height:.2f} cm"
        else:
            height_text = f"Height: {pixel_height} pixels"

        # Add text label
        cv2.putText(annotated_image, height_text, (mid_x + 10, (ceiling_y + floor_y) // 2),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    return annotated_image

In [11]:
import os
import re
import cv2
import numpy as np

def natural_sort_key(filename):
    """Extract numerical values from filenames for correct sorting."""
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', filename)]

# Define folders
image_folder = "Input_test_folder(3m approx)"
output_folder = "Output_test_folder_combined1"
propeller_output_folder = "Output_test_folder_propeller_only2"

os.makedirs(output_folder, exist_ok=True)
os.makedirs(propeller_output_folder, exist_ok=True)

# Load models
coke_model = YOLO("best (8).pt")
floor_model = YOLO("best (5).pt")
propeller_model = YOLO("best (9).pt")

ceil_heights = {}  # Store detected ceiling heights
Propeller_heights = {}
image_results = []

# Get and sort image filenames
image_files = sorted(
    [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))],
    key=natural_sort_key
)
i = 0
for image_name in image_files:
    image_path = os.path.join(image_folder, image_name)
    # Detect Coca-Cola bottle height (for calibration)
    height_pixels, estimated_real_height = detect_height_bottle(image_path, coke_model)
    calibration_obj = (estimated_real_height, height_pixels) if height_pixels else None
    # Detect ceiling and floor
    floor_results, image = detect_objects(floor_model, image_path)
    # Perform inference
    ceiling_y, floor_y = get_ceiling_floor_coordinates(floor_results,floor_model)
    if ceiling_y is None or floor_y is None:
        ceiling_height = "XXXXX"
    elif calibration_obj:
        ceiling_height = calculate_height(ceiling_y, floor_y, calibration_obj)
    else:
        ceiling_height = "XXXXX"
    
    print(f"Ceiling Height for {image_name}: {ceiling_height}")
    
    if ceiling_height != "XXXXX":
        ceil_heights[image_name] = ceiling_height
    # Read the image
    image = cv2.imread(image_path)
    # Detect propellers (AC)
    propeller_results, _ = detect_objects(propeller_model, image_path)
    propeller_y_bottoms, propeller_boxes = get_propeller_coordinates(propeller_results)
    propeller_heights_details = []
    propeller_heights_d = []
    if propeller_boxes and calibration_obj:
        for idx, propeller_y_bottom in enumerate(propeller_y_bottoms):
            real_height = calculate_height(propeller_y_bottom, floor_y, calibration_obj)
            propeller_heights_details.append(
                f"P{idx + 1}: {real_height:.2f} cm"
            )
            propeller_heights_d.append(real_height.item())
    if propeller_heights_d != []:
        Propeller_heights[image_name] = propeller_heights_d
    ac_height = ", ".join(propeller_heights_details) if propeller_heights_details else "YYYY"
    print(f"AC Height for {image_name}: {ac_height}")
    if ceiling_y is not None and floor_y is not None and ceiling_height != "XXXXX":
        mid_x = image.shape[1] // 2
        cv2.line(image, (mid_x, ceiling_y), (mid_x, floor_y), (0, 255, 0), 3)
        text_position = (mid_x + 10, (ceiling_y + floor_y) // 2)
        cv2.putText(image, f"{ceiling_height:.2f} cm", text_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
    # Draw propeller detections
    if calibration_obj:
        for idx, (box, real_height) in enumerate(zip(propeller_boxes, propeller_heights_d)):
            x1, y1, x2, y2 = box
            label = f"P{idx + 1}: {real_height:.2f} cm"
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw bounding box
            cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)  # Draw correct label
    
        output_path = os.path.join(output_folder, image_name)
        cv2.imwrite(output_path, image)
        print(f"Output saved at: {output_path}")

        # If propellers are detected, save in a separate folder
        if propeller_boxes:
            propeller_output_path = os.path.join(propeller_output_folder, image_name)
            cv2.imwrite(propeller_output_path, image)
            print(f"Propeller Output saved at: {propeller_output_path}")
    image_results.append(f"Photo {i+1} | ID {image_name} | Ceiling height {ceiling_height} | AC Height {ac_height}")
    i += 1

# Print final results
print("\nFinal Results:")
for result in image_results:
    print(result)



0: 800x608 1 750ml coke, 969.4ms
Speed: 13.1ms preprocess, 969.4ms inference, 14.7ms postprocess per image at shape (1, 3, 800, 608)

0: 640x480 2 ceilings, 2 floors, 4 wallss, 793.5ms
Speed: 14.0ms preprocess, 793.5ms inference, 29.5ms postprocess per image at shape (1, 3, 640, 480)
Ceiling Height for image1.jpeg: 311.1754455566406

0: 800x608 (no detections), 915.0ms
Speed: 11.9ms preprocess, 915.0ms inference, 1.5ms postprocess per image at shape (1, 3, 800, 608)
AC Height for image1.jpeg: YYYY
Output saved at: Output_test_folder_combined\image1.jpeg

0: 800x608 1 750ml coke, 863.9ms
Speed: 12.8ms preprocess, 863.9ms inference, 3.6ms postprocess per image at shape (1, 3, 800, 608)

0: 640x480 5 floors, 2 wallss, 767.1ms
Speed: 8.9ms preprocess, 767.1ms inference, 12.4ms postprocess per image at shape (1, 3, 640, 480)
Ceiling Height for image2.jpeg: XXXXX

0: 800x608 (no detections), 905.3ms
Speed: 9.3ms preprocess, 905.3ms inference, 1.9ms postprocess per image at shape (1, 3, 800,

In [12]:
print("\nFinal Dictionary:")
for key, value in Propeller_heights.items():
    print(f"{key}: {value}")


Final Dictionary:
image3.jpeg: [175.87408447265625]
image4.jpeg: [219.10037231445312]
image5.jpeg: [204.6835174560547, 269.3392333984375, 298.2735290527344, 253.62181091308594]
image7.jpeg: [207.71853637695312, 168.1038818359375]
image8.jpeg: [303.2469787597656]
image9.jpeg: [195.7265625, 157.0364227294922, 212.30804443359375]
image10.jpeg: [259.8856506347656, 254.76113891601562, 236.09329223632812, 240.8517608642578, 225.8442840576172]
image11.jpeg: [190.6743621826172, 194.9712371826172, 194.4341278076172, 67.94452667236328]
image13.jpeg: [163.06947326660156]
image14.jpeg: [265.4924011230469]
image24.jpeg: [220.6201934814453, 249.59405517578125]
image25.jpeg: [259.6839599609375]
image28.jpeg: [222.73057556152344]
image29.jpeg: [276.1257019042969, 261.1905822753906]
image30.jpeg: [243.8071746826172]
image31.jpeg: [296.6328430175781, 267.5846862792969]
image32.jpeg: [239.8228759765625]
image33.jpeg: [286.1464538574219]
image35.jpeg: [303.5136413574219, 264.21209716796875]
image36.jpeg:

In [13]:
Actual_P_heights = {
    "image3.jpeg": [250],
    "image4.jpeg": [200],
    "image5.jpeg": [200, 280, 280, 280],
    "image7.jpeg": [280, 180],
    "image8.jpeg": [280],
    "image9.jpeg": [280, 180, 280],
    "image10.jpeg": [280, 280, 200, 200, 180],
    "image11.jpeg": [230, 230, 230, 80],
    "image13.jpeg": [280],
    "image14.jpeg": [280],
    "image24.jpeg": [280, 280],
    "image25.jpeg": [280],
    "image28.jpeg": [280],
    "image29.jpeg": [280, 280],
    "image30.jpeg": [280],
    "image31.jpeg": [280, 280],
    "image32.jpeg": [280],
    "image33.jpeg": [280],
    "image35.jpeg": [280, 280],
    "image36.jpeg": [280, 280, 280],
    "image37.jpeg": [280, 280],
    "image40.jpeg": [200, 200],
    "image45.jpeg": [280],
    "image52.jpeg": [280],
    "image53.jpeg": [280]
}


In [14]:
# Flatten the lists to compare elements one by one
predicted_P_heights = []
real_P_heights = []

for img in Propeller_heights:
    if img in Actual_P_heights:
        predicted_P_heights.extend(Propeller_heights[img])
        real_P_heights.extend(Actual_P_heights[img])

# Convert to NumPy arrays
predicted_P_heights = np.array(predicted_P_heights)
real_heights = np.array(real_P_heights)

# Compute evaluation metrics
mae = np.mean(np.abs(real_P_heights - predicted_P_heights))  # Mean Absolute Error
mse = np.mean((real_P_heights - predicted_P_heights) ** 2)  # Mean Squared Error
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Print results
formatted_P_heights = ", ".join(f"{h:.2f}" for h in predicted_P_heights)
form_real_P_heights = ", ".join(f"{h:.2f}" for h in real_P_heights)
print("\n")
print(f"Predicted Heights: [{formatted_P_heights}]")
print("\n")
print(f"Actual Heights: [{form_real_P_heights}]")
print(f"MAE: {mae:.2f} cm")
print(f"MSE: {mse:.2f} cm²")
print(f"RMSE: {rmse:.2f} cm")



Predicted Heights: [175.87, 219.10, 204.68, 269.34, 298.27, 253.62, 207.72, 168.10, 303.25, 195.73, 157.04, 212.31, 259.89, 254.76, 236.09, 240.85, 225.84, 190.67, 194.97, 194.43, 67.94, 163.07, 265.49, 220.62, 249.59, 259.68, 222.73, 276.13, 261.19, 243.81, 296.63, 267.58, 239.82, 286.15, 303.51, 264.21, 306.72, 245.12, 196.22, 263.50, 284.89, 207.11, 208.48, 261.20, 150.39, 228.65]


Actual Heights: [250.00, 200.00, 200.00, 280.00, 280.00, 280.00, 280.00, 180.00, 280.00, 280.00, 180.00, 280.00, 280.00, 280.00, 200.00, 200.00, 180.00, 230.00, 230.00, 230.00, 80.00, 280.00, 280.00, 280.00, 280.00, 280.00, 280.00, 280.00, 280.00, 280.00, 280.00, 280.00, 280.00, 280.00, 280.00, 280.00, 280.00, 280.00, 280.00, 280.00, 280.00, 200.00, 200.00, 280.00, 280.00, 280.00]
MAE: 34.35 cm
MSE: 1982.39 cm²
RMSE: 44.52 cm


In [15]:
# Calculate Mean Absolute Percentage Error (MAPE)
mape = np.mean(np.abs((real_P_heights - predicted_P_heights) / real_P_heights)) * 100

# Calculate Accuracy
accuracy = 100 - mape

# Print results
print(f"MAPE: {mape:.2f}%")
print(f"Accuracy: {accuracy:.2f}%") 

MAPE: 13.47%
Accuracy: 86.53%


In [16]:
import numpy as np
import torch

# Predicted height (assumed constant for all images as per your requirement)
actual_height = 340  # cm

# Ensure all predicted heights are converted to CPU NumPy arrays
predicted_heights = [
    h.detach().cpu().numpy() if isinstance(h, torch.Tensor) else np.array(h, dtype=np.float32)
    for h in ceil_heights.values()
]

# Convert to NumPy array
predicted_heights = np.array(predicted_heights, dtype=np.float32)

# Create an array of the actual height for comparison
real_heights = np.full_like(predicted_heights, actual_height)

# Compute evaluation metrics
mae = np.mean(np.abs(real_heights - predicted_heights))  # Mean Absolute Error
mse = np.mean((real_heights - predicted_heights) ** 2)  # Mean Squared Error
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Print results
formatted_heights = ", ".join(f"{h:.2f}" for h in predicted_heights)
print(f"Predicted Heights: [{formatted_heights}]")
print(f"MAE: {mae:.2f} cm")
print(f"MSE: {mse:.2f} cm²")
print(f"RMSE: {rmse:.2f} cm")

Predicted Heights: [311.18, 308.03, 331.98, 372.93, 356.02, 325.18, 302.37, 383.61, 318.51, 303.21, 304.46, 326.38, 308.69, 288.98, 267.66, 292.85, 276.66, 326.78, 275.78, 278.08, 297.95, 305.32, 318.85, 325.56, 252.21, 248.38, 293.42, 347.77, 255.22, 341.93, 297.93, 348.32, 365.94, 270.90, 329.96, 318.41, 332.52, 316.81, 265.65, 312.59, 334.41, 281.95, 254.67, 341.89, 204.28, 260.30]
MAE: 39.88 cm
MSE: 2447.36 cm²
RMSE: 49.47 cm


In [17]:
# # Find the accuracy of the propellers

# import numpy as np

# # Predicted height (assumed constant for all images as per your requirement)
# actual_height = 340  # cm

# # Extract actual heights from the provided results
# # image_heights = {
# #     "image2.jpeg": 197.32,
# #     "image5.jpeg": 210.09,
# #     "image4.jpeg": 196.10,
# #     "image1.jpeg": 120.91,
# # }

# # List to store actual heights dynamically
# predicted_heights = []
# real_heights = []

# # Fetch actual heights using a loop
# # Convert actual heights to a NumPy array
# predicted_heights = np.array(list(ceil_heights.values()))

# # Create an array of the actual height for comparison
# real_heights = np.full_like(predicted_heights, actual_height)

# # Convert to NumPy arrays for calculations
# # actual_heights = np.array(actual_heights)
# # predicted_heights = np.full_like(actual_heights, predicted_height)

# # Compute evaluation metrics
# mae = np.mean(np.abs(real_heights - predicted_heights))  # Mean Absolute Error
# mse = np.mean((real_heights - predicted_heights) ** 2)  # Mean Squared Error
# rmse = np.sqrt(mse)  # Root Mean Squared Error

# # Print results
# formatted_heights = ", ".join(f"{h:.2f}" for h in predicted_heights)
# print(f"Predicted Heights: [{formatted_heights}]")
# print(f"MAE: {mae:.2f} cm")
# print(f"MSE: {mse:.2f} cm²")
# print(f"RMSE: {rmse:.2f} cm")

In [18]:
# Calculate Mean Absolute Percentage Error (MAPE)
mape = np.mean(np.abs((real_heights - predicted_heights) / real_heights)) * 100

# Calculate Accuracy
accuracy = 100 - mape

# Print results
print(f"MAPE: {mape:.2f}%")
print(f"Accuracy: {accuracy:.2f}%") 

MAPE: 11.73%
Accuracy: 88.27%


In [19]:
import numpy as np
import os

# Define actual height (constant for all images)
actual_height = 340  # cm

# Ensure all predicted heights are converted to CPU NumPy arrays
# predicted_heights = [
#     h.detach().cpu().numpy() if isinstance(h, torch.Tensor) else np.array(h, dtype=np.float32)
#     for h in ceil_heights.values()
# ]

# # Convert to NumPy array
# predicted_heights = np.array(predicted_heights, dtype=np.float32)
# # Convert predicted heights to NumPy array
# predicted_heights = np.array(list(ceil_heights.values()))

# Create an array of actual height values
real_heights = np.full_like(predicted_heights, actual_height)

# Calculate metrics
mae = np.mean(np.abs(real_heights - predicted_heights))  # Mean Absolute Error
mse = np.mean((real_heights - predicted_heights) ** 2)  # Mean Squared Error
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Define folder and file path
folder_path = "Output_test_folder_combined"
os.makedirs(folder_path, exist_ok=True)
file_path = os.path.join(folder_path, "Ceiling_Height_file.txt")

# Save results to a text file
with open(file_path, "w") as file:
    file.write("Image Name | Actual Height (cm) | Predicted Height (cm) | Error (cm)\n")
    file.write("-" * 70 + "\n")
    
    for image_name, predicted_height in ceil_heights.items():
        error = abs(actual_height - predicted_height)
        file.write(f"{image_name} |     {actual_height} cm     |     {predicted_height:.2f} cm     |     {error:.2f} cm    \n")
    


print(f"Results saved in: {file_path}")

Results saved in: Output_test_folder_combined\Ceiling_Height_file.txt


In [20]:
# import os

# # Define folder and file path
# folder_path1 = "Output_test_folder_propeller_only"
# os.makedirs(folder_path1, exist_ok=True)
# file_path = os.path.join(folder_path1, "propeller_Height_file.txt")




# # Write to file
# with open(file_path, "w") as file:
#     # Header
#     file.write("Image Name |AC_Prop1|AC_Prop2|AC_Prop3|AC_Prop4|AC_Prop5\n")
#     file.write("-" * 75 + "\n")
    
#     for image_name, heights in sorted(Propeller_heights.items()):
#         formatted_heights = [f"{h:.2f}cm" for h in heights]  # Format to 2 decimal places
#         while len(formatted_heights) < 5:
#             formatted_heights.append("YYYYYYYY")  # Fill missing columns with '-'
        
#         file.write(f"{image_name}|" + "|".join(formatted_heights)+"\n")

# print(f"✅ Formatted propeller heights saved in: {file_path}")


In [21]:
import os
import re

# Define folder and file path
folder_path1 = "Output_test_folder_propeller_only"
os.makedirs(folder_path1, exist_ok=True)
file_path = os.path.join(folder_path1, "propeller_Height_file.txt")

# Function to sort image names in human-readable order (image1, image2, ..., image50)
def sort_key(image_name):
    match = re.search(r'(\d+)', image_name)
    return int(match.group(1)) if match else float('inf')

# Sort image names properly
sorted_image_names = sorted(Propeller_heights.keys(), key=sort_key)

with open(file_path, "w") as file:
    # Header
    file.write("Image Name | AC_Prop1 | AC_P_Prop1 | AC_Prop2 | AC_P_Prop2 | AC_Prop3 | AC_P_Prop3 | AC_Prop4 | AC_P_Prop4 | AC_Prop5 | AC_P_Prop5 \n")
    file.write("-" * 95 + "\n")
    
    for image_name in sorted_image_names:
        actual_heights = Actual_P_heights.get(image_name, [])
        predicted_heights = Propeller_heights.get(image_name, [])
        
        formatted_actual = [f"{h:.2f}cm" for h in actual_heights]
        formatted_predicted = [f"{h:.2f}cm" for h in predicted_heights]
        
        while len(formatted_actual) < 5:
            formatted_actual.append("YYYYYYYY")  # Placeholder for missing values
        while len(formatted_predicted) < 5:
            formatted_predicted.append("YYYYYYYY")
        
        row_values = [val for pair in zip(formatted_actual, formatted_predicted) for val in pair]
        file.write(f"{image_name}| " + " | ".join(row_values) + "\n")

# Print results
print(f"✅ Formatted propeller heights saved in: {file_path}")
print(f"MAE: {mae:.2f} cm")
print(f"MSE: {mse:.2f} cm²")
print(f"RMSE: {rmse:.2f} cm")


✅ Formatted propeller heights saved in: Output_test_folder_propeller_only\propeller_Height_file.txt
MAE: 39.88 cm
MSE: 2447.36 cm²
RMSE: 49.47 cm


In [22]:
# Write to file
import os

# Define folder and file path
folder_path1 = "Output_test_folder_propeller_only"
os.makedirs(folder_path1, exist_ok=True)
file_path = os.path.join(folder_path1, "propeller_Height_file.txt")
with open(file_path, "w") as file:
    # Header
    file.write("Image Name | AC_Prop1 | AC_P_Prop1 | AC_Prop2 | AC_P_Prop2 | AC_Prop3 | AC_P_Prop3 | AC_Prop4 | AC_P_Prop4 | AC_Prop5 | AC_P_Prop5 \n")
    file.write("-" * 95 + "\n")
    
    
    for image_name in sorted(Propeller_heights.keys()):
        actual_heights = Actual_P_heights.get(image_name, [])
        predicted_heights = Propeller_heights.get(image_name, [])
        
        formatted_actual = [f"{h:.2f}cm" for h in actual_heights]
        formatted_predicted = [f"{h:.2f}cm" for h in predicted_heights]
        
        while len(formatted_actual) < 5:
            formatted_actual.append("YYYYYYYY")  # Placeholder for missing values
        while len(formatted_predicted) < 5:
            formatted_predicted.append("YYYYYYYY")
        
        row_values = [val for pair in zip(formatted_actual, formatted_predicted) for val in pair]
        file.write(f"{image_name}| " + " | ".join(row_values) + "\n")



# Print results
print(f"✅ Formatted propeller heights saved in: {file_path}")
print(f"MAE: {mae:.2f} cm")
print(f"MSE: {mse:.2f} cm²")
print(f"RMSE: {rmse:.2f} cm")


✅ Formatted propeller heights saved in: Output_test_folder_propeller_only\propeller_Height_file.txt
MAE: 39.88 cm
MSE: 2447.36 cm²
RMSE: 49.47 cm
